<a href="https://colab.research.google.com/github/arshad831/1-Langchain/blob/main/4_API_Calling_omantel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install requests

1. Setting up Autorization
2. add the end point url
3. add the body

In [3]:
import os

from google.colab import userdata


groq_api_key = userdata.get("GROQ_API_KEY")

2. Calling API Using Python

In [4]:
import requests

# 1 Define the URL for the Groq API endpoint
url = "https://api.groq.com/openai/v1/chat/completions"

#2  Set the headers for the API request
headers = {
    "Authorization": f"Bearer {groq_api_key}"
}

# 3 Define the body for the API request
body = {
    "model": "llama-3.1-8b-instant",
    "messages": [
        {
            "role": "user",
            "content": "What is artificial general intelligence?"
        }
    ]
}

# Send a POST request to the Groq API
response = requests.post(url, headers=headers, json=body)

# Check if the request was successful
if response.status_code == 200:
    print(response.json()['choices'][0]['message']['content'])
else:
    print("Error:", response.json())


Artificial General Intelligence (AGI) refers to a type of artificial intelligence (AI) that possesses the ability to understand, learn, and apply knowledge across a wide range of tasks, similar to human intelligence. It's a hypothetical AI system that can perform any intellectual task that a human being can, at least to the level of a human expert.

AGI is often characterized by the following properties:

1. **Autonomy**: AGI can operate without direct human supervision and can adapt to new situations.
2. **Reasoning**: AGI can reason abstractly and draw inferences from incomplete or uncertain information.
3. **Learning**: AGI can learn from its experiences, memories, and interactions with its environment.
4. **Problem-solving**: AGI can identify and solve complex problems, often in a multi-step process.
5. **Knowledge representation**: AGI can represent and integrate knowledge from multiple domains.
6. **Reasoning and thinking**: AGI can exhibit human-like reasoning, creativity, and c

Adding Omantel Customization

In [11]:
import os
import requests
import gradio as gr



url = "https://api.groq.com/openai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {groq_api_key}",
    "Content-Type": "application/json"
}


LOGO_URL = "https://raw.githubusercontent.com/Decoding-Data-Science/Omantel/main/download%20(36).png"

OMANTEL_SYSTEM = """You are Aisha, the Omantel AI assistant for developer enablement.

Style
- Be concise, professional, and action-oriented.
- Structure all answers as: Summary → Steps → Code → Notes.
- Prefer bullet points over long paragraphs.

Domain
- Focus on telecom/customer-care use cases: RAG over KB articles, intent classification, call/chat summarization, ticket triage, PII redaction, and evaluation.
- Show Python examples (requests, FastAPI, Gradio) and Groq (Llama 3.1-8B).
- Use realistic ops/QA metrics when relevant: FRT, AHT, intent accuracy, containment/deflection rate, CSAT, latency (p95), and token usage.

Safety & Privacy
- Do not invent facts; say “I’m not sure” if uncertain.
- Never include real keys, internal URLs, or customer data. Use placeholders.
- Redact PII patterns in examples (MSISDN, IMSI, national IDs, emails).
- Assume all data is confidential; follow least-privilege principles.

Localization
- Default to English. If the user writes in Arabic, reply in Arabic.
- Use Asia/Muscat timezone for dates/times when needed.

Formatting
- Put code in fenced blocks with minimal, runnable examples.
- Include curl when helpful.
- Add clear TODOs for env vars and secrets.
"""

def call_groq(messages, temperature=0.2, max_tokens=600, model="llama-3.1-8b-instant"):
    body = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    resp = requests.post(url, headers=headers, json=body, timeout=60)
    if resp.status_code == 200:
        return resp.json()["choices"][0]["message"]["content"]
    else:
        try:
            return f"Error: {resp.status_code} {resp.json()}"
        except Exception:
            return f"Error: {resp.status_code} {resp.text}"

def groq_chat(message, history):
    messages = [{"role": "system", "content": OMANTEL_SYSTEM}]
    for u, a in history:
        if u:
            messages.append({"role": "user", "content": u})
        if a:
            messages.append({"role": "assistant", "content": a})
    messages.append({"role": "user", "content": message})
    return call_groq(messages)

# --- Minimal UI ---
with gr.Blocks(title="Omantel Developer Assistant") as app:
    # Small CSS to keep logo left and title vertically centered
    gr.HTML("""
    <style>
      .header-row { display:flex; align-items:center; gap:12px; }
      .header-logo img { max-height:48px; border-radius:8px; }
      .header-title { font-size:20px; font-weight:600; margin:0; }
    </style>
    """)

    with gr.Row(elem_classes=["header-row"]):
        logo = gr.Image(value=LOGO_URL, label=None, show_label=False, interactive=False, height=100, elem_classes=["header-logo"])
        title_box = gr.Textbox(value="Omantel Developer Assistant (Aisha)", label="Title", lines=1)
        header_md = gr.Markdown("### Omantel Developer Assistant (Aisha)", elem_classes=["header-title"])

    # Update displayed header when title changes
    def _update_header(t):
        return f"### {t}" if t.strip() else "### Omantel Developer Assistant (Aisha)"
    title_box.change(_update_header, inputs=title_box, outputs=header_md)

    gr.Markdown("Ask about telecom/customer-care AI use cases. Answers follow **Summary → Steps → Code → Notes**.")

    gr.ChatInterface(
        fn=groq_chat,
        title=None,  # we already show our own header
        description=None,
        textbox=gr.Textbox(placeholder="Ask about RAG over FAQs, intent classification, summarization, redaction, eval metrics...", lines=2),
        cache_examples=False,
    )

if __name__ == "__main__":
    app.launch()


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a1cf38b4fd053220e4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install gradio

In [7]:
import os
import requests
import gradio as gr



url = "https://api.groq.com/openai/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {groq_api_key}",
    "Content-Type": "application/json"
}

OMANTEL_SYSTEM = """You are Aisha, the Omantel AI assistant for developer enablement.

Style
- Be concise, professional, and action-oriented.
- Structure all answers as: Summary → Steps → Code → Notes.
- Prefer bullet points over long paragraphs.

Domain
- Focus on telecom/customer-care use cases: RAG over KB articles, intent classification, call/chat summarization, ticket triage, PII redaction, and evaluation.
- Show Python examples (requests, FastAPI, Gradio) and Groq (Llama 3.1-8B).
- Use realistic ops/QA metrics when relevant: FRT, AHT, intent accuracy, containment/deflection rate, CSAT, latency (p95), and token usage.

Safety & Privacy
- Do not invent facts; say “I’m not sure” if uncertain.
- Never include real keys, internal URLs, or customer data. Use placeholders.
- Redact PII patterns in examples (MSISDN, IMSI, national IDs, emails).
- Assume all data is confidential; follow least-privilege principles.

Localization
- Default to English. If the user writes in Arabic, reply in Arabic.
- Use Asia/Muscat timezone for dates/times when needed.

Formatting
- Put code in fenced blocks with minimal, runnable examples.
- Include curl when helpful.
- Add clear TODOs for env vars and secrets.
"""

# --- Core API call as a function for reuse ---
def call_groq(messages, temperature=0.2, max_tokens=600, model="llama-3.1-8b-instant"):
    body = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    resp = requests.post(url, headers=headers, json=body, timeout=60)
    if resp.status_code == 200:
        return resp.json()["choices"][0]["message"]["content"]
    else:
        try:
            return f"Error: {resp.status_code} {resp.json()}"
        except Exception:
            return f"Error: {resp.status_code} {resp.text}"

# --- Gradio chat handler (maps Gradio history -> OpenAI messages) ---
def groq_chat(message, history, temperature=0.2, max_tokens=600, model="llama-3.1-8b-instant"):
    messages = [{"role": "system", "content": OMANTEL_SYSTEM}]
    # history: list of [user_msg, bot_msg]
    for u, a in history:
        if u:
            messages.append({"role": "user", "content": u})
        if a:
            messages.append({"role": "assistant", "content": a})
    # current user turn
    messages.append({"role": "user", "content": message})
    return call_groq(messages, temperature=temperature, max_tokens=max_tokens, model=model)

# --- Optional: a quick test call you had (kept for parity) ---
def demo_single_call():
    test_body = [
        {"role": "system", "content": OMANTEL_SYSTEM},
        {"role": "user", "content": "Draft a minimal RAG plan for Omantel care FAQs and list the key evaluation metrics."}
    ]
    print(call_groq(test_body))

# --- Gradio UI ---
with gr.Blocks(title="Omantel Developer Assistant") as app:
    gr.Markdown("### Omantel Developer Assistant (Aisha)\nAsk about telecom/customer-care AI use cases, code, and evaluation.")
    with gr.Row():
        model = gr.Dropdown(
            choices=["llama-3.1-8b-instant"],
            value="llama-3.1-8b-instant",
            label="Model"
        )
        temperature = gr.Slider(0.0, 1.0, value=0.2, step=0.05, label="Temperature")
        max_tokens = gr.Slider(128, 4096, value=600, step=32, label="Max Tokens")

    chat = gr.ChatInterface(
        fn=lambda message, history: groq_chat(
            message, history,
            temperature=float(temperature.value) if hasattr(temperature, "value") else 0.2,
            max_tokens=int(max_tokens.value) if hasattr(max_tokens, "value") else 600,
            model=str(model.value) if hasattr(model, "value") else "llama-3.1-8b-instant"
        ),
        title="Chat with Aisha",
        description="Concise, actionable answers with Summary → Steps → Code → Notes.",
        textbox=gr.Textbox(placeholder="Ask about RAG over FAQs, intent classifiers, redaction, eval metrics, etc.", lines=2),
        theme="default",
        cache_examples=False,
        submit_btn="Send",
        clear_btn="Clear"
    )

if __name__ == "__main__":
    # demo_single_call()  # uncomment to run the one-off call in console
    app.launch()


TypeError: ChatInterface.__init__() got an unexpected keyword argument 'clear_btn'

In [ ]:
#for hugging face  u need to set env seperately and save a app.py file

import os
import requests
import gradio as gr

# Retrieve the API key from the environment variable
groq_api_key = os.getenv("GROQ_API_KEY")


# Define the API endpoint and headers
url = "https://api.groq.com/openai/v1/chat/completions"
headers = {"Authorization": f"Bearer {groq_api_key}"}

# Function to interact with Groq API
def chat_with_groq(user_input):
    body = {
        "model": "llama-3.1-8b-instant",
        "messages": [{"role": "user", "content": user_input}]
    }

    response = requests.post(url, headers=headers, json=body)

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error: {response.json()}"

# Create Gradio interface
interface = gr.Interface(
    fn=chat_with_groq,
    inputs=gr.Textbox(lines=2, placeholder="Ask me anything..."),
    outputs=gr.Textbox(),
    title="Chat with Groq AI (Llama 3.1-8B)",
    description="Type your question below and get a response powered by Groq's Llama 3.1-8B model."
)

# Launch Gradio app
if __name__ == "__main__":
    interface.launch()



In [ ]:
##prompt template


import os
import requests
from google.colab import userdata  # Secure storage for API keys in Colab

# Get the Groq API key securely (use userdata in Colab)
groq_api_key = userdata.get("GROQ_API_KEY")  # Store using: userdata.set("GROQ_API_KEY", "your_api_key")

if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found! Set it using userdata.set('GROQ_API_KEY', 'your_api_key')")

# Define the Groq API endpoint
url = "https://api.groq.com/openai/v1/chat/completions"

# Set headers for the API request
headers = {
    "Authorization": f"Bearer {groq_api_key}"
}

# Define a prompt template
prompt_template = """You are a helpful assistant with expertise in Telecomunication
User Query: {query}
Please provide a detailed, beginner-friendly response."""

# Function to generate a response using the Groq API
def ask_groq(query):
    prompt = prompt_template.format(query=query)

    body = {
        "model": "llama-3.1-8b-instant",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }

    response = requests.post(url, headers=headers, json=body)

    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return f"Error: {response.status_code} - {response.text}"


In [ ]:
response_text = ask_groq("What is different plans in omantel?")
print(response_text)


Omantel is a popular telecommunications service provider in Oman, offering a range of plans to suit different needs and budgets. As a helpful assistant, I'd be happy to break down the different plans offered by Omantel into easy-to-understand categories.

**1. Prepaid Plans**

Prepaid plans are a great option for those who want to pay as they go. With Omantel's prepaid plans, you can top up your credit and use it to make calls, send texts, and access data. Here are a few prepaid plans:

- **Daily Plans**: These plans offer a limited amount of internet and calling minutes for a day. For example, the Daily Unlimited Plan costs RO 1 (approximately $2.60 USD) per day and includes 1 GB of data and unlimited minutes.
- **Weekly Plans**: These plans offer more data and calling minutes for a week. For example, the Weekly Unlimited Plan costs RO 6 (approximately $15.60 USD) per week and includes 5 GB of data and unlimited minutes.
- **Monthly Plans**: These plans offer even more data and callin